In [64]:
from google.cloud import bigquery
from google.cloud import storage
import os
import json
import pandas as pd
import requests
from datetime import datetime, timedelta

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "leadsreporting-680714dc15e1.json"

In [3]:
bq = bigquery.Client()

### Read and format the historical data from Rory's CDK export

In [70]:
df = pd.read_csv("../2_data/Historical Leads 1-1-2017_4-24-2019.csv", encoding="unicode_escape")
len(df.columns), len(df)

(44, 338446)

In [6]:
sorted(df.columns)

['Actual  Response Time (Minutes)',
 'Ad Source',
 'Address',
 'Appointment Set',
 'Appointment Show',
 'Assigned CSR',
 'Back Gross',
 'Birth Year',
 'City',
 'Customer-Id',
 'Date/Time In',
 'Days to Sale',
 'Deal No',
 'Deal-Id',
 'Front Gross',
 'Gender',
 'Location',
 'Location State',
 'Logged By User',
 'Lost Sale Reason',
 'OEM',
 'Original Event Date',
 'Prospect Type',
 'Provider',
 'Purchase Method',
 'Sale Type',
 'Sales Manager',
 'Sales Person 2',
 'Salesperson',
 'Selling Price',
 'Sold Date',
 'Source',
 'State',
 'Status',
 'Stock ',
 'Stock Type',
 'Sub-status',
 'Total Gross',
 'TradeIn-Year-Make-Model',
 'Vehicle',
 'Vehicle Make',
 'Vehicle Model',
 'Vehicle Year',
 'Zip']

In [7]:
# format datetime fields for BQ
def prep_date(col, out_format="%Y-%m-%d %H:%M:%S"):
    d_clean = pd.to_datetime(col, errors='ignore')
    return d_clean.apply(lambda x: x.strftime(out_format))
df['Date/Time In'] = prep_date(df['Date/Time In'])
df['Original Event Date'] = prep_date(df['Original Event Date'])

In [14]:
# format date fields for BQ
def clean_date(date):
    try:
        return datetime.strptime(str(date), "%m/%d/%Y").strftime("%Y-%m-%d")
    except:
        return None
df['Sold Date'] = clean_date(df['Sold Date'])

In [15]:
df.drop(labels = ['Actual  Response Time (Minutes)',
                  'City', 
                  'State', 
                  'Zip', 
                  'OEM', 
                  'Location State', 
                  'Gender', 
                  'Address'], axis=1, inplace=True)

In [16]:
col_mapping = {'Ad Source': 'ad_source',
            'Appointment Set': 'appt_set',
            'Appointment Show': 'appt_show',
            'Assigned CSR': 'assigned_csr',
            'Back Gross': 'back_gross',
            'Customer-Id': 'client_id',
            'Birth Year': 'birth_year',
            'Date/Time In': 'date_time_in',
            'Days to Sale': 'days_to_sale',
            'Deal-Id': 'deal_id',
            'Deal No': 'deal_num',
            'Front Gross': 'front_gross',
            'Location': 'location',
            'Logged By User': 'logged_by_user',
            'Lost Sale Reason': 'lost_sale_reason',
            'Original Event Date': 'original_event_date',
            'Prospect Type': 'prospect_type',
            'Provider': 'provider',
            'Purchase Method': 'purchase_method',
            'Sale Type': 'sale_type',
            'Sales Manager': 'sales_manager',
            'Salesperson': 'sales_person',
            'Sales Person 2': 'sales_person_2',
            'Selling Price': 'selling_price',
            'Sold Date': 'sold_date',
            'Source': 'source',
            'Status': 'status',
            'Stock ': 'stock',
            'Stock Type': 'stock_type',
            'Sub-status': 'sub_status',
            'Total Gross': 'total_gross',
            'TradeIn-Year-Make-Model': 'tradein_vehicle',
            'Vehicle': 'vehicle',
            'Vehicle Make': 'vehicle_make',
            'Vehicle Model': 'vehicle_model',
            'Vehicle Year': 'vehicle_year'}

df.rename(index=str, columns = col_mapping, inplace=True)

In [17]:
# create fake last_action_date field for testing
df['last_action_date'] = df ['date_time_in']

In [18]:
df.head().T

,0,1,2,3,4
status,Lost Sale,Delivered,Lost Sale,Lost Sale,Lost Sale
date_time_in,2017-06-30 23:53:00,2017-06-30 23:51:00,2017-06-30 23:44:00,2017-06-30 23:34:00,2017-06-30 23:24:00
stock,NaN,H5175967,LXG006708,M25607,M25691
stock_type,New,New,Used,Used,Used
vehicle,2017 Mercedes-Benz Metris Passenger Van,2017 Lexus GX460,2016 Lexus RX350,2014 Jeep Wrangler SPORT,2010 Toyota Corolla
vehicle_year,2017,2017,2016,2014,2010
vehicle_make,Mercedes-Benz,Lexus,Lexus,Jeep,Toyota
vehicle_model,Metris Passenger Van,GX460,RX350,Wrangler,Corolla
source,Inbound Internet,Inbound Internet,Inbound Internet,Inbound Internet,Inbound Internet
sales_person,"Bedford, Corey","Elam, Josh","Herring, Rob","Massey, Randy","Massey, Randy"


In [19]:
# save just the first 1k rows for testing
df.iloc[0:1000].to_csv("../2_data/Historical Leads 1-1-2017_4-24-2019 - CLEAN-Sample.csv", index=False)

### Upload formatted test csv data to cloud storage

In [23]:
gcs = storage.Client()

In [24]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print('File {} uploaded to {}.'.format(
        source_file_name,
        destination_blob_name))

In [25]:
upload_blob('leads-reporting',
            '../2_data/Historical Leads 1-1-2017_4-24-2019 - CLEAN-Sample.csv',
            'leads_historical.csv')

File ../2_data/Historical Leads 1-1-2017_4-24-2019 - CLEAN-Sample.csv uploaded to leads_historical.csv.


### Read in schema file that was origininally output from BQ and then updated locally

In [51]:
with open('leads_table_schema.json', 'r') as f:
    schema = json.loads(f.read())
schema

[{'mode': 'NULLABLE', 'name': 'status', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'date_time_in', 'type': 'TIMESTAMP'},
 {'mode': 'NULLABLE', 'name': 'stock', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'stock_type', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'vehicle', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'vehicle_year', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'vehicle_make', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'vehicle_model', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'source', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'sales_person', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'original_event_date', 'type': 'TIMESTAMP'},
 {'mode': 'NULLABLE', 'name': 'sub_status', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'lost_sale_reason', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'deal_num', 'type': 'FLOAT'},
 {'mode': 'NULLABLE', 'name': 'ad_source', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'prospect_typ

In [47]:
# make sure schema and columns are in same order
for i in range(len(schema)):
    print(schema[i]['name'], '  -  ', df.columns[i])

status   -   status
date_time_in   -   date_time_in
stock   -   stock
stock_type   -   stock_type
vehicle   -   vehicle
vehicle_year   -   vehicle_year
vehicle_make   -   vehicle_make
vehicle_model   -   vehicle_model
source   -   source
sales_person   -   sales_person
original_event_date   -   original_event_date
sub_status   -   sub_status
lost_sale_reason   -   lost_sale_reason
deal_num   -   deal_num
ad_source   -   ad_source
prospect_type   -   prospect_type
sales_person_2   -   sales_person_2
sales_manager   -   sales_manager
sale_type   -   sale_type
logged_by_user   -   logged_by_user
assigned_csr   -   assigned_csr
tradein_vehicle   -   tradein_vehicle
sold_date   -   sold_date
purchase_method   -   purchase_method
front_gross   -   front_gross
back_gross   -   back_gross
total_gross   -   total_gross
birth_year   -   birth_year
appt_set   -   appt_set
appt_show   -   appt_show
location   -   location
provider   -   provider
days_to_sale   -   days_to_sale
deal_id   -   deal_i

### Delete the table if it exists (clean-up during testing)

In [52]:
table_id = 'leadsreporting.leads_testing.leads_historical'
bq.delete_table(table_id, not_found_ok=True)
print("Deleted table '{}'.".format(table_id))

Deleted table 'leadsreporting.leads_testing.leads_historical'.


### Create new table and load from the rest csv using the imported schema

In [53]:
dataset_ref = bq.dataset('leads_testing')
job_config = bigquery.LoadJobConfig()
job_config.schema = [ bigquery.SchemaField(i["name"], i["type"]) for i in schema ]
job_config.skip_leading_rows = 1
job_config.source_format = bigquery.SourceFormat.CSV
uri = "gs://leads-reporting/leads_historical.csv"

In [54]:
load_job = bq.load_table_from_uri(
    uri, dataset_ref.table("leads_historical"), job_config=job_config
)  # API request
print("Starting job {}".format(load_job.job_id))

load_job.result()  # Waits for table load to complete.
print("Job finished.")

destination_table = bq.get_table(dataset_ref.table("leads_historical"))
print("Loaded {} rows.".format(destination_table.num_rows))

Starting job 0bcd41fd-aa90-4898-b4f8-4a963921f880
Job finished.
Loaded 1000 rows.


### Query the BQ table to get last lead/modified datetime

In [57]:
query_job = bq.query("""
    SELECT 
        max(date_time_in) AS max_date_time_in, 
        max(last_action_date) AS max_last_action_date
    FROM `leadsreporting`.leads_testing.leads_historical""")

results = query_job.result()
start_times = results.to_dataframe()

In [63]:
print('NEW       date_from  - ', start_times.iloc[0]['max_date_time_in'].strftime('%Y-%d-%dT%H:%M:%S'))
print('MODIFIED  date_from  - ', start_times.iloc[0]['max_last_action_date'].strftime('%Y-%d-%dT%H:%M:%S'))

NEW       date_from  -  2017-30-30T23:53:00
MODIFIED  date_from  -  2017-30-30T23:53:00


### Pull some sample rows to inspect the BQ table

In [56]:
sample_rows = bq.query("""
    SELECT *
    FROM `leadsreporting`.leads_testing.leads_historical
    LIMIT 10""")

results = sample_rows.result()
results.to_dataframe()

,status,date_time_in,stock,stock_type,vehicle,vehicle_year,vehicle_make,vehicle_model,source,sales_person,...,birth_year,appt_set,appt_show,location,provider,days_to_sale,deal_id,client_id,selling_price,last_action_date
0,Lost Sale,2017-06-30 22:33:00+00:00,None,Used,None,None,None,None,Inbound Internet,"Massey, Randy",...,NaN,False,False,Park Place Motorcars Dallas,Edmunds Direct,None,1727587,1737564.0,0.0,2017-06-30 22:33:00+00:00
1,Lost Sale,2017-06-30 19:18:00+00:00,None,None,None,None,None,None,Inbound Internet,"Van Sicklen, Jeff",...,1980.0,False,False,Park Place Porsche,Cobalt - ContactUs,None,1727592,1538406.0,0.0,2017-06-30 19:18:00+00:00
2,Lost Sale,2017-06-30 19:08:00+00:00,None,Used,None,None,None,None,Outbound Phone Call,"Mushtaq, Naseem",...,NaN,False,False,Park Place Lexus Plano,None,None,1727506,1064202.0,0.0,2017-06-30 19:08:00+00:00
3,Lost Sale,2017-06-30 18:40:00+00:00,None,Used,None,None,None,None,Inbound Phone Call,"Lopez, Marco",...,1984.0,False,False,Park Place Motorcars Arlington,None,None,1727496,962915.0,0.0,2017-06-30 18:40:00+00:00
4,Lost Sale,2017-06-30 18:36:00+00:00,None,None,None,None,None,None,Inbound Internet,"Mushtaq, Naseem",...,NaN,False,False,Park Place Lexus Plano,Cobalt Website,None,1727494,1693800.0,0.0,2017-06-30 18:36:00+00:00
5,Lost Sale,2017-06-30 17:16:00+00:00,None,New,None,None,None,None,Showroom,"Simmons, Brian",...,1967.0,False,False,Park Place Rolls Bentley,None,None,1727473,28191.0,0.0,2017-06-30 17:16:00+00:00
6,Lost Sale,2017-06-30 16:49:00+00:00,None,Used,None,None,None,None,Inbound Phone Call,"Holyfield, Winston",...,NaN,False,False,Park Place Motorcars Dallas,None,None,1727453,1818188.0,0.0,2017-06-30 16:49:00+00:00
7,Lost Sale,2017-06-30 16:32:00+00:00,None,Used,None,None,None,None,Inbound Phone Call,"Booker, Cadarrel",...,NaN,False,False,Park Place Motorcars Dallas,None,None,1727443,1818182.0,0.0,2017-06-30 16:32:00+00:00
8,Lost Sale,2017-06-30 16:15:00+00:00,None,Used,None,None,None,None,Inbound Phone Call,"Booker, Cadarrel",...,1969.0,False,False,Park Place Motorcars Dallas,None,None,1727435,341516.0,0.0,2017-06-30 16:15:00+00:00
9,Lost Sale,2017-06-30 16:13:00+00:00,None,Used,None,None,None,None,Showroom,"Ali, Adi",...,1966.0,False,False,Park Place Lexus Plano,None,None,1727432,1818174.0,0.0,2017-06-30 16:13:00+00:00


### Pull new records from API

In [65]:
# create some fake timestamps for testing
start_date = (datetime.today() - timedelta(days=3)).strftime('%Y-%m-%dT%H:%M:%S')
end_date = (datetime.today() - timedelta(days=2)).strftime('%Y-%m-%dT%H:%M:%S')

payload = {'date_from': start_date, 
           'date_to': end_date,
           'update': 'false'}

print(payload)

r = requests.get('https://api.parkplace.com/leads/new', params=payload)
r.status_code

{'date_from': '2019-04-23T08:07:55', 'date_to': '2019-04-24T08:07:55', 'update': 'false'}


200

In [67]:
leads = r.json()
api = pd.DataFrame(leads['Leads'])
print(min(api['original_event_date']), max(api['original_event_date']))
print(min(api['date_time_in']), max(api['date_time_in']))
print(min(api['last_action_date']), max(api['last_action_date']))

2019-01-15 01:19:00 2019-04-24 12:29:08
2019-04-23 01:00:16 2019-04-24 12:29:08
2019-01-15 01:19:00 2019-04-24 12:29:08


In [68]:
len(api.columns) #API has more columns now ... need to update BQ table and schema

46

In [69]:
sorted(api.columns)

['actual_response_time',
 'ad_source',
 'address',
 'appointment_set',
 'appointment_show',
 'assigned_csr',
 'back_gross',
 'birth_year',
 'city',
 'client_id',
 'date_time_in',
 'days_to_sale',
 'deal_id',
 'deal_num',
 'fi_manager',
 'front_gross',
 'gender',
 'has_duplicates',
 'last_action_date',
 'lead_id',
 'location',
 'logged_by_user',
 'lost_sale_reason',
 'original_event_date',
 'prospect_type',
 'provider',
 'purchase_method',
 'sale_type',
 'sales_manager',
 'sales_person',
 'sales_person_2',
 'selling_price',
 'sold_date',
 'source',
 'state',
 'status',
 'stock',
 'stock_type',
 'sub_status',
 'total_gross',
 'tradein_vehicle',
 'vehicle',
 'vehicle_make',
 'vehicle_model',
 'vehicle_year',
 'zip']

### Insert new records from API into BQ table - TBD

### Run BQ de-duplication query - TBD